In [0]:
import requests
r = requests.get('https://zenodo.org/api/records/1188976')


In [3]:
!wget https://zenodo.org/api/files/c8f9b6fe-82ac-481c-ad9c-12b5581cb4b4/Audio_Song_Actors_01-24.zip
!wget https://zenodo.org/api/files/c8f9b6fe-82ac-481c-ad9c-12b5581cb4b4/Audio_Speech_Actors_01-24.zip

!unzip -q -d ./song Audio_Song_Actors_01-24.zip 
!unzip -q -d ./speech Audio_Speech_Actors_01-24.zip 


--2020-05-07 16:26:10--  https://zenodo.org/api/files/c8f9b6fe-82ac-481c-ad9c-12b5581cb4b4/Audio_Song_Actors_01-24.zip
Resolving zenodo.org (zenodo.org)... 188.184.117.155
Connecting to zenodo.org (zenodo.org)|188.184.117.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 225505317 (215M) [application/octet-stream]
Saving to: ‘Audio_Song_Actors_01-24.zip’

Audio_Song_Actors_0 100%[===================>] 215.06M  9.55MB/s    in 24s     

2020-05-07 16:26:36 (9.03 MB/s) - ‘Audio_Song_Actors_01-24.zip’ saved [225505317/225505317]

--2020-05-07 16:26:39--  https://zenodo.org/api/files/c8f9b6fe-82ac-481c-ad9c-12b5581cb4b4/Audio_Speech_Actors_01-24.zip
Resolving zenodo.org (zenodo.org)... 188.184.117.155
Connecting to zenodo.org (zenodo.org)|188.184.117.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 208468073 (199M) [application/octet-stream]
Saving to: ‘Audio_Speech_Actors_01-24.zip’

Audio_Speech_Actors 100%[===================>] 198

In [0]:
with open("download.txt", "w+") as f:
    for file in r.json()["files"]:
        aLink=(file["links"]["self"])
        f.write(aLink+"\n")

In [0]:
!sudo apt-get install -y aria2


In [0]:
!aria2c -i download.txt -d ./dataset

In [0]:
!unzip "/content/SharedDatasets/dataset/*" -d ./unzipped/

In [0]:
!du -hs ./dataset

24G	./dataset


In [0]:
unzip "datazipped/*" -d ./unzipped/

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!mkdir unzipped
!cp -r ./drive/My\ Drive/SharedDatasets/unzipped/ ./unzipped


Modality (01 = full-AV, 02 = video-only, 03 = audio-only).

Vocal channel (01 = speech, 02 = song).

Emotion (01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised).

Emotional intensity (01 = normal, 02 = strong). NOTE: There is no strong intensity for the 'neutral' emotion.

Statement (01 = "Kids are talking by the door", 02 = "Dogs are sitting by the door").

Repetition (01 = 1st repetition, 02 = 2nd repetition).

Actor (01 to 24. Odd numbered actors are male, even numbered actors are female).

In [0]:
import glob
file_list=glob.glob("./speech/**/*.wav")+glob.glob("./song/**/*.wav")


In [23]:
fname,part

('./song/Actor_21/03-02-04-01-02-02-21.wav',
 ['03', '02', '04', '01', '02', '02', '21'])

In [36]:

import os
import pandas as pd

emotion = []
gender = []
path = []
all_files=[]
for wavFile in file_list:
    fname = wavFile
    
    part = fname.split('/')[-1][:-4].split('-')
    # print(part)
    emotion.append(int(part[2]))

    temp = int(part[6])
    if temp%2 == 0:
        temp = "female"
    else:
        temp = "male"
    gender.append(temp)
    path.append(fname.split("/")[-1])

        
RAV_df = pd.DataFrame(emotion)
RAV_df = RAV_df.replace({1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'})
RAV_df = pd.concat([pd.DataFrame(gender),RAV_df],axis=1)
RAV_df.columns = ['gender','emotion']
RAV_df['labels'] =RAV_df.gender + '_' + RAV_df.emotion
RAV_df['source'] = 'RAVDESS'  
RAV_df = pd.concat([RAV_df,pd.DataFrame(path, columns = ['path'])],axis=1)
# RAV_df = RAV_df.drop(['gender', 'emotion'], axis=1)
RAV_df.labels.value_counts()


male_sad           192
male_calm          192
male_fear          192
male_happy         192
male_angry         192
female_angry       184
female_fear        184
female_sad         184
female_happy       184
female_calm        184
male_disgust        96
male_neutral        96
male_surprise       96
female_surprise     96
female_disgust      96
female_neutral      92
Name: labels, dtype: int64

In [37]:
RAV_df

,gender,emotion,labels,source,path
0,male,happy,male_happy,RAVDESS,03-01-03-02-01-02-05.wav
1,male,angry,male_angry,RAVDESS,03-01-05-02-02-02-05.wav
2,male,fear,male_fear,RAVDESS,03-01-06-02-02-02-05.wav
3,male,sad,male_sad,RAVDESS,03-01-04-02-02-01-05.wav
4,male,surprise,male_surprise,RAVDESS,03-01-08-02-01-01-05.wav
...,...,...,...,...,...
2447,male,angry,male_angry,RAVDESS,03-02-05-02-01-01-21.wav
2448,male,fear,male_fear,RAVDESS,03-02-06-01-02-02-21.wav
2449,male,happy,male_happy,RAVDESS,03-02-03-01-01-02-21.wav
2450,male,fear,male_fear,RAVDESS,03-02-06-02-02-01-21.wav


In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test\
    = train_test_split(RAV_df["path"], RAV_df["emotion"], test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val\
    = train_test_split(X_train, y_train, test_size=0.25, random_state=42)


In [0]:
import csv

with open('train.csv', mode='w') as employee_file:
    employee_writer = csv.writer(employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for i in zip(X_train,y_train):
        employee_writer.writerow(i)

with open('test.csv', mode='w') as employee_file:
    employee_writer = csv.writer(employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for i in zip(X_test,y_test):
        employee_writer.writerow(i)

with open('valid.csv', mode='w') as employee_file:
    employee_writer = csv.writer(employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for i in zip(X_val,y_val):
        employee_writer.writerow(i)


	zip warning: missing end signature--probably not a zip file (did you
	zip warning: remember to use binary mode when you transferred it?)
	zip warning: (if you are trying to read a damaged archive try -F)

zip error: Zip file structure invalid (test.csv)


In [40]:
len(y_test),len(y_train),len(y_val)

(491, 1470, 491)

In [41]:
from collections import Counter
Counter(y_test)


Counter({'angry': 74,
         'calm': 78,
         'disgust': 32,
         'fear': 78,
         'happy': 64,
         'neutral': 36,
         'sad': 77,
         'surprise': 52})

In [31]:
Counter(y_val)


Counter({'angry': 74,
         'calm': 76,
         'disgust': 43,
         'fear': 75,
         'happy': 88,
         'neutral': 42,
         'sad': 67,
         'surprise': 26})

In [32]:
Counter(y_train)


Counter({'angry': 228,
         'calm': 222,
         'disgust': 117,
         'fear': 223,
         'happy': 224,
         'neutral': 110,
         'sad': 232,
         'surprise': 114})